In [3]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import time
from torchvision.utils import save_image
import os

In [4]:
apply_transform = transforms.Compose([transforms.Resize([256,256]),transforms.ToTensor()])
apply_transform0 = transforms.Compose([transforms.Resize([256,256]),transforms.ToTensor()])
apply_transform1 = transforms.Compose([transforms.Resize([128,128]),transforms.ToTensor()])
apply_transform2 = transforms.Compose([transforms.Resize([64,64]),transforms.ToTensor()])
apply_transform3 = transforms.Compose([transforms.Resize([32,32]),transforms.ToTensor()])

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')

GPU is available!


In [3]:
image_path = '/home/cvgws2/juniors_blur/dataset_arch_3/image'
ground_path = '/home/cvgws2/juniors_blur/dataset_arch_3/gt'
train_ = []
print(len(os.listdir(image_path)))
for i in os.listdir(image_path):
    ii , iii = map(str,i.split('.'))
    a = Image.open(os.path.join(image_path,i))
    a = apply_transform(a)
    x = Image.open(os.path.join(ground_path,ii+'.png'))
    b = apply_transform(x)
    c = apply_transform1(x)
    d = apply_transform2(x)
    e = apply_transform3(x)
    train_.append([a,b,c,d,e])

979


In [0]:
image_path = '/content/gdrive/My Drive/Colab Notebooks/dataset/Nonblur'
ground_path = '/content/gdrive/My Drive/Colab Notebooks/dataset/image_screenshot_17.11.2019.png'
for i in os.listdir(image_path):
    ii , iii = map(str,i.split('.'))
    a = Image.open(os.path.join(image_path,i))
    a = apply_transform(a)
    x = Image.open(ground_path)
    b = apply_transform(x)
    c = apply_transform1(x)
    d = apply_transform2(x)
    e = apply_transform3(x)
    train_.append([a,b,c,d,e])

In [4]:
import random
random.shuffle(train_)

In [5]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.ReLU()
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)


In [6]:
class BasicConvTranspose2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConvTranspose2d, self).__init__()
        self.conv = nn.ConvTranspose2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.ReLU()
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [7]:
class NN_model(nn.Module):
    def __init__(self):
        super(NN_model,self).__init__()
        self.conv1=nn.Sequential(BasicConv2d(3,16,kernel_size=3,padding=1,stride=2))

        self.conv2=nn.Sequential(BasicConv2d(16,32,kernel_size=3,padding=1,stride=2))
        #self.maxpool=nn.MaxPool2d(2,2)
        self.conv3=nn.Sequential(BasicConv2d(32,64,kernel_size=3,padding=1,stride=2))
        #self.maxpool=nn.MaxPool2d(2,2)
        self.conv4=nn.Sequential(BasicConv2d(64,128,kernel_size=3,padding=1,stride=2))

        self.deconv1=nn.Sequential(BasicConvTranspose2d(128,64,kernel_size=3,padding=1,output_padding = 1,stride=2))

        self.deconv2=nn.Sequential(BasicConvTranspose2d(64,32,kernel_size=3,padding=1,output_padding = 1,stride=2)) 
        
        self.deconv3=nn.Sequential(BasicConvTranspose2d(32,16,kernel_size=3,padding=1,output_padding = 1,stride=2))
        
        self.deconv4=nn.Sequential(BasicConvTranspose2d(16,3,kernel_size=3,padding=1,output_padding = 1,stride=2)) 
        
        self.conv1x1_1= nn.Conv2d(16,1,kernel_size = 1,padding=0,stride = 1)
        self.conv1x1_2= nn.Conv2d(32,1,kernel_size = 1,padding=0,stride = 1)
        self.conv1x1_3= nn.Conv2d(64,1,kernel_size = 1,padding=0,stride = 1)
        self.conv1x1_4= nn.Conv2d(128,1,kernel_size = 1,padding=0,stride = 1)
        self.conv1x1_o1= nn.Conv2d(3,1,kernel_size = 1,padding=0,stride = 1)
        
    def forward(self,x):
        r1 = self.conv1(x)
        r2 = self.conv2(r1)
        r3 = self.conv3(r2)
        x = self.conv4(r3)
        
        r1 = self.conv1x1_1(r1)
        r2 = self.conv1x1_2(r2)
        r3 = self.conv1x1_3(r3)
        r4 = self.conv1x1_4(x)
        x = x+r4
        r4 = self.deconv1(x)
        x = r4 + r3
        r3 = self.deconv2(x)
        x = r3+r2
        r2 = self.deconv3(x)
        x = r2+r1
        r1 = self.deconv4(x)
        r1 = self.conv1x1_o1(r1)
        r2 = self.conv1x1_1(r2)
        r3 = self.conv1x1_2(r3)
        r4 = self.conv1x1_3(r4)
        return r1,r2,r3,r4

haha = NN_model()
if use_gpu:
    haha = haha.cuda()
print(haha)

NN_model(
  (conv1): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): ReLU()
    )
  )
  (conv2): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): ReLU()
    )
  )
  (conv3): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): ReLU()
    )
  )
  (conv4): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): ReLU()
    )
  )
  (deconv1): Sequential(
    (0): BasicConvTranspose2d(
      (conv): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1), bias=False)
      (bn): ReLU()
    )
  )
  (deconv2): Sequential(
    (0): BasicConvTranspose2d(
      (conv): ConvTranspose2d(64, 32, kernel_size=(3, 3), st

In [9]:
trainLoader = torch.utils.data.DataLoader(train_, batch_size=10,shuffle=True, num_workers=2)

In [8]:
haha = torch.load('./blur1.pth')

In [ ]:
optimizer = optim.Adam(haha.parameters(),lr = 1e-3)
criterion = nn.MSELoss()

In [12]:
def train(epoch):
    train_loss = 0
    start = time.time()
    for i in train_:
        data,gt1,gt2,gt3,gt4 = i
        if use_gpu:
            data = data.cuda()
            gt1 = gt1.cuda()
            gt2 = gt2.cuda()
            gt3 = gt3.cuda()
            gt4 = gt4.cuda()

        optimizer.zero_grad()
        o1,o2,o3,o4= haha(data.unsqueeze(0))
        loss = criterion(o1,gt1.unsqueeze(0)) 
        loss += criterion(o2,gt2.unsqueeze(0))
        loss += criterion(o3,gt3.unsqueeze(0))
        loss += criterion(o4,gt4.unsqueeze(0))
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    endt = time.time() - start
    print('====> Epoch: {} Average loss: {:.4f} Time : {:.0f}m {:.0f}s'.format(epoch, train_loss/len(train_),endt//60,endt%60))
    train_losses.append(train_loss / (len(train_)))
    return train_loss / (len(train_))
    

In [15]:
train_losses = []
iterations = 500
for i in range(iterations):
    train_losses.append(train(i))
torch.save(haha,'./blur1.pth')

====> Epoch: 0 Average loss: 0.0514 Time : 0m 13s
====> Epoch: 1 Average loss: 0.0518 Time : 0m 13s
====> Epoch: 2 Average loss: 0.0506 Time : 0m 13s
====> Epoch: 3 Average loss: 0.0499 Time : 0m 10s
====> Epoch: 4 Average loss: 0.0503 Time : 0m 9s
====> Epoch: 5 Average loss: 0.0516 Time : 0m 12s
====> Epoch: 6 Average loss: 0.0496 Time : 0m 13s
====> Epoch: 7 Average loss: 0.0491 Time : 0m 11s
====> Epoch: 8 Average loss: 0.0505 Time : 0m 13s
====> Epoch: 9 Average loss: 0.0515 Time : 0m 14s
====> Epoch: 10 Average loss: 0.0510 Time : 0m 13s
====> Epoch: 11 Average loss: 0.0527 Time : 0m 15s
====> Epoch: 12 Average loss: 0.0491 Time : 0m 15s
====> Epoch: 13 Average loss: 0.0497 Time : 0m 14s
====> Epoch: 14 Average loss: 0.0498 Time : 0m 15s
====> Epoch: 15 Average loss: 0.0524 Time : 0m 15s
====> Epoch: 16 Average loss: 0.0496 Time : 0m 12s
====> Epoch: 17 Average loss: 0.0501 Time : 0m 15s
====> Epoch: 18 Average loss: 0.0500 Time : 0m 14s
====> Epoch: 19 Average loss: 0.0502 Time 

====> Epoch: 161 Average loss: 0.0481 Time : 0m 15s
====> Epoch: 162 Average loss: 0.0475 Time : 0m 15s
====> Epoch: 163 Average loss: 0.0504 Time : 0m 15s
====> Epoch: 164 Average loss: 0.0478 Time : 0m 15s
====> Epoch: 165 Average loss: 0.0482 Time : 0m 15s
====> Epoch: 166 Average loss: 0.0482 Time : 0m 15s
====> Epoch: 167 Average loss: 0.0480 Time : 0m 15s
====> Epoch: 168 Average loss: 0.0488 Time : 0m 15s
====> Epoch: 169 Average loss: 0.0488 Time : 0m 15s
====> Epoch: 170 Average loss: 0.0481 Time : 0m 15s
====> Epoch: 171 Average loss: 0.0502 Time : 0m 15s
====> Epoch: 172 Average loss: 0.0471 Time : 0m 15s
====> Epoch: 173 Average loss: 0.0470 Time : 0m 15s
====> Epoch: 174 Average loss: 0.0504 Time : 0m 15s
====> Epoch: 175 Average loss: 0.0505 Time : 0m 11s
====> Epoch: 176 Average loss: 0.0480 Time : 0m 10s
====> Epoch: 177 Average loss: 0.0483 Time : 0m 13s
====> Epoch: 178 Average loss: 0.0474 Time : 0m 15s
====> Epoch: 179 Average loss: 0.0470 Time : 0m 12s
====> Epoch:

====> Epoch: 319 Average loss: 0.0465 Time : 0m 10s
====> Epoch: 320 Average loss: 0.0464 Time : 0m 10s
====> Epoch: 321 Average loss: 0.0486 Time : 0m 10s
====> Epoch: 322 Average loss: 0.0466 Time : 0m 10s
====> Epoch: 323 Average loss: 0.0462 Time : 0m 9s
====> Epoch: 324 Average loss: 0.0484 Time : 0m 10s
====> Epoch: 325 Average loss: 0.0460 Time : 0m 10s
====> Epoch: 326 Average loss: 0.0462 Time : 0m 10s
====> Epoch: 327 Average loss: 0.0480 Time : 0m 10s
====> Epoch: 328 Average loss: 0.0476 Time : 0m 10s
====> Epoch: 329 Average loss: 0.0469 Time : 0m 10s
====> Epoch: 330 Average loss: 0.0456 Time : 0m 10s
====> Epoch: 331 Average loss: 0.0482 Time : 0m 9s
====> Epoch: 332 Average loss: 0.0484 Time : 0m 10s
====> Epoch: 333 Average loss: 0.0461 Time : 0m 10s
====> Epoch: 334 Average loss: 0.0451 Time : 0m 10s
====> Epoch: 335 Average loss: 0.0472 Time : 0m 10s
====> Epoch: 336 Average loss: 0.0495 Time : 0m 10s
====> Epoch: 337 Average loss: 0.0454 Time : 0m 10s
====> Epoch: 3

====> Epoch: 478 Average loss: 0.0471 Time : 0m 11s
====> Epoch: 479 Average loss: 0.0464 Time : 0m 15s
====> Epoch: 480 Average loss: 0.0447 Time : 0m 11s
====> Epoch: 481 Average loss: 0.0459 Time : 0m 9s
====> Epoch: 482 Average loss: 0.0471 Time : 0m 9s
====> Epoch: 483 Average loss: 0.0475 Time : 0m 9s
====> Epoch: 484 Average loss: 0.0448 Time : 0m 10s
====> Epoch: 485 Average loss: 0.0441 Time : 0m 9s
====> Epoch: 486 Average loss: 0.0469 Time : 0m 9s
====> Epoch: 487 Average loss: 0.0450 Time : 0m 9s
====> Epoch: 488 Average loss: 0.0444 Time : 0m 10s
====> Epoch: 489 Average loss: 0.0477 Time : 0m 15s
====> Epoch: 490 Average loss: 0.0462 Time : 0m 15s
====> Epoch: 491 Average loss: 0.0468 Time : 0m 14s
====> Epoch: 492 Average loss: 0.0450 Time : 0m 13s
====> Epoch: 493 Average loss: 0.0461 Time : 0m 12s
====> Epoch: 494 Average loss: 0.0458 Time : 0m 12s
====> Epoch: 495 Average loss: 0.0459 Time : 0m 14s
====> Epoch: 496 Average loss: 0.0478 Time : 0m 10s
====> Epoch: 497 A

In [16]:
path = '/home/cvgws2/juniors_blur/test'
for file in os.listdir(path):
    image = Image.open(os.path.join(path,file))
    image = apply_transform(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)
    if use_gpu:
        inputs = Variable(image).cuda()
    else:
        inputs = Variable(image)
    recon_,_,_,_ = haha(inputs)
    comparison = recon_.view(1, 1, 256, 256)[0]
    save_image(comparison.cpu(),os.path.join('/home/cvgws2/juniors_blur/results',file))

In [19]:
image = Image.open('/home/cvgws2/juniors_blur/images1.jpeg')
image = apply_transform(image).float()
image = Variable(image, requires_grad=True)
image = image.unsqueeze(0)
if use_gpu:
    inputs = Variable(image).cuda()
else:
    inputs = Variable(image)
recon_,_,_,_ = haha(inputs)
comparison = recon_.view(1, 1, 256, 256)[0]
save_image(comparison.cpu(),'/home/cvgws2/juniors_blur/trial1.png')